In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

In [ ]:
df = pd.read_csv("/content/csv for model.csv")

In [ ]:
# Remove those rows in which have  label is not mentioned
df = df.dropna(subset=['label'])
df['label'] = df['label'].astype(int)
# Get the indices of the last 50 rows with label 0
indices_to_remove = df[df['label'] == 0].index[-50:]

# Remove those rows from the DataFrame
df= df.drop(indices_to_remove)

text = df.content.values
labels = df.label.values


<ipython-input-38-285f981a6d2d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['label'].astype(int)


In [ ]:
# Count the occurrences of each class in the 'label' column
label_counts = df['label'].value_counts()

# Print the counts
print(label_counts)

0    111
1    107
Name: label, dtype: int64


In [ ]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
    )

In [ ]:
def print_rand_sentence():
  '''Displays the tokens and respective IDs of a random text sample'''
  index = random.randint(0, len(text)-1)
  table = np.array([tokenizer.tokenize(text[index]), 
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
  print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

╒════════════╤═════════════╕
│ Tokens     │   Token IDs │
╞════════════╪═════════════╡
│ anything   │        2505 │
├────────────┼─────────────┤
│ happens    │        6433 │
├────────────┼─────────────┤
│ in         │        1999 │
├────────────┼─────────────┤
│ that       │        2008 │
├────────────┼─────────────┤
│ five       │        2274 │
├────────────┼─────────────┤
│ minutes    │        2781 │
├────────────┼─────────────┤
│ and        │        1998 │
├────────────┼─────────────┤
│ i          │        1045 │
├────────────┼─────────────┤
│ '          │        1005 │
├────────────┼─────────────┤
│ m          │        1049 │
├────────────┼─────────────┤
│ yours      │        6737 │
├────────────┼─────────────┤
│ no         │        2053 │
├────────────┼─────────────┤
│ matter     │        3043 │
├────────────┼─────────────┤
│ what       │        2054 │
├────────────┼─────────────┤
│ happens    │        6433 │
├────────────┼─────────────┤
│ a          │        1037 │
├────────────┼

In [ ]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 32,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )


for sample in text:
  encoding_dict = preprocessing(sample, tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
token_id[6]


tensor([  101,  1031,  7702, 21418,  1033,  2086,  1012,  1012,  1012,  1031,
         5503,  6159,  1033,  2053,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0])

In [ ]:
def print_rand_sentence_encoding():
  '''Displays tokens, token IDs and attention mask of a random text sample'''
  index = random.randint(0, len(text) - 1)
  tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
  token_ids = [i.numpy() for i in token_id[index]]
  attention = [i.numpy() for i in attention_masks[index]]

  table = np.array([tokens, token_ids, attention]).T
  print(tabulate(table, 
                 headers = ['Tokens', 'Token IDs', 'Attention Mask'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence_encoding()

╒════════════╤═════════════╤══════════════════╕
│ Tokens     │   Token IDs │   Attention Mask │
╞════════════╪═════════════╪══════════════════╡
│ [CLS]      │         101 │                1 │
├────────────┼─────────────┼──────────────────┤
│ assaulted  │       17536 │                1 │
├────────────┼─────────────┼──────────────────┤
│ her        │        2014 │                1 │
├────────────┼─────────────┼──────────────────┤
│ it         │        2009 │                1 │
├────────────┼─────────────┼──────────────────┤
│ '          │        1005 │                1 │
├────────────┼─────────────┼──────────────────┤
│ s          │        1055 │                1 │
├────────────┼─────────────┼──────────────────┤
│ not        │        2025 │                1 │
├────────────┼─────────────┼──────────────────┤
│ good       │        2204 │                1 │
├────────────┼─────────────┼──────────────────┤
│ enough     │        2438 │                1 │
├────────────┼─────────────┼────────────

In [ ]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels)

# Train and validation sets
train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx], 
                        attention_masks[val_idx], 
                        labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

In [ ]:
def b_tp(preds, labels):
  '''Returns True Positives (TP): count of correct predictions of actual class 1'''
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  '''
  Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
  '''
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity

In [ ]:
# Load the BertForSequenceClassification model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5,
                              eps = 1e-08
                              )

# Run on GPU
#model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 7

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')


Epoch:  14%|█▍        | 1/7 [01:28<08:51, 88.54s/it]


	 - Train loss: 0.7099
	 - Validation Accuracy: 0.5278
	 - Validation Precision: 0.4667
	 - Validation Recall: 0.2910
	 - Validation Specificity: 0.7672



Epoch:  29%|██▊       | 2/7 [02:50<07:01, 84.40s/it]


	 - Train loss: 0.6553
	 - Validation Accuracy: 0.5208
	 - Validation Precision: 0.5222
	 - Validation Recall: 0.4101
	 - Validation Specificity: 0.6270



Epoch:  43%|████▎     | 3/7 [04:07<05:24, 81.14s/it]

...


	 - Train loss: 0.1948
	 - Validation Accuracy: 0.6389
	 - Validation Precision: 0.6667
	 - Validation Recall: 0.5026
	 - Validation Specificity: 0.7672



Epoch:  86%|████████▌ | 6/7 [07:58<01:18, 78.63s/it]


	 - Train loss: 0.0908
	 - Validation Accuracy: 0.5556
	 - Validation Precision: 0.5417
	 - Validation Recall: 0.6905
	 - Validation Specificity: 0.4206



Epoch: 100%|██████████| 7/7 [09:16<00:00, 79.50s/it]


	 - Train loss: 0.0198
	 - Validation Accuracy: 0.5486
	 - Validation Precision: 0.5476
	 - Validation Recall: 0.5608
	 - Validation Specificity: 0.5423



In [ ]:
from sklearn.metrics import accuracy_score
def predict(texts, model, tokenizer):
    inputs = tokenizer(texts, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)
    predicted_classes = torch.argmax(probabilities, dim=-1)
    return predicted_classes

In [ ]:
X_test=df['content'].tolist()

In [ ]:
X_test=X_test[:50]

In [ ]:
len(X_test)

50

In [ ]:
predictions = predict(X_test, model, tokenizer)
predictions = predictions.detach().numpy()

In [ ]:
y_test = df['label'].tolist()

In [ ]:
y_test=y_test[:50]

In [ ]:
test_accuracy = accuracy_score(y_test, predictions)
print(f"Testing accuracy: {test_accuracy:.4f}")

Testing accuracy: 0.9200


In [ ]:
import pandas as pd
from transformers import pipeline
from sklearn.metrics import accuracy_score

# Load your DataFrame
df = df.sample(70)
# Initialize the zero-shot classification pipeline
zero_shot_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Define your label candidates
labels = [0, 1]

# Function to predict label using the zero-shot classifier
def predict_label(text):
    result = zero_shot_classifier(text, labels)
    predicted_label = result['labels'][0]
    return 1 if predicted_label == "threat" else 0

# Apply the prediction function to the 'text' column of the DataFrame
df['predicted_label'] = df['content'].apply(predict_label)

# Calculate accuracy
accuracy = accuracy_score(df['label'], df['predicted_label'])

print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 48.57%
